# Quick Start in 30s

In [ ]:
# install this if you areusing colab
! pip install leann

## Build the index

In [1]:
from leann.api import LeannBuilder

builder = LeannBuilder(backend_name="hnsw")
builder.add_text("C# is a powerful programming language and it is good at game development")
builder.add_text("Python is a powerful programming language and it is good at machine learning tasks")
builder.add_text("Machine learning transforms industries")
builder.add_text("Neural networks process complex data")
builder.add_text("Leann is a great storage saving engine for RAG on your MacBook")
builder.build_index("knowledge.leann")

INFO: Registering backend 'hnsw'


/Users/yichuan/Desktop/code/LEANN/leann/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: facebook/contriever
Batches: 100%|██████████| 1/1 [00:00<00:00, 13.51it/s]
INFO:leann_backend_hnsw.hnsw_backend:INFO: Converting HNSW index to CSR-pruned format...


M: 64 for level: 0
Starting conversion: knowledge.index -> knowledge.csr.tmp
[0.00s] Reading Index HNSW header...
[0.00s]   Header read: d=768, ntotal=5
[0.00s] Reading HNSW struct vectors...
  Reading vector (dtype=<class 'numpy.float64'>, fmt='d')... Count=6, Bytes=48
[0.00s]   Read assign_probas (6)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=7, Bytes=28
[0.11s]   Read cum_nneighbor_per_level (7)
  Reading vector (dtype=<class 'numpy.int32'>, fmt='i')... Count=5, Bytes=20
[0.21s]   Read levels (5)
[0.30s]   Probing for compact storage flag...
[0.30s]   Found compact flag: False
[0.30s]   Compact flag is False, reading original format...
[0.30s]   Probing for potential extra byte before non-compact offsets...
[0.30s]   Found and consumed an unexpected 0x00 byte.
  Reading vector (dtype=<class 'numpy.uint64'>, fmt='Q')... Count=6, Bytes=48
[0.30s]   Read offsets (6)
[0.40s]   Attempting to read neighbors vector...
  Reading vector (dtype=<class 'numpy.int32'>, fmt

INFO:leann_backend_hnsw.hnsw_backend:✅ CSR conversion successful.
INFO:leann_backend_hnsw.hnsw_backend:INFO: Replaced original index with CSR-pruned version at 'knowledge.index'


## Search with real-time embeddings

In [2]:
from leann.api import LeannSearcher

searcher = LeannSearcher("knowledge.leann")
results = searcher.search("programming languages", top_k=2)
results

INFO:leann.api:🔍 LeannSearcher.search() called:
INFO:leann.api:  Query: 'programming languages'
INFO:leann.api:  Top_k: 2
INFO:leann.api:  Additional kwargs: {}
INFO:leann.embedding_server_manager:Port 5557 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Port 5558 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Port 5559 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Using port 5560 instead of 5557
INFO:leann.embedding_server_manager:Starting embedding server on port 5560...
INFO:leann.embedding_server_manager:Command: /Users/yichuan/Desktop/code/LEANN/leann/.venv/bin/python -m leann_backend_hnsw.hnsw_embedding_server --zmq-port 5560 --model-name facebook/contriever --passages-file knowledge.leann.meta.json
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can eith

[read_HNSW - CSR NL v4] Reading metadata & CSR indices (manual offset)...
[read_HNSW NL v4] Read levels vector, size: 5
[read_HNSW NL v4] Reading Compact Storage format indices...
[read_HNSW NL v4] Read compact_level_ptr, size: 10
[read_HNSW NL v4] Read compact_node_offsets, size: 6
[read_HNSW NL v4] Read entry_point: 4, max_level: 0
[read_HNSW NL v4] Read storage fourcc: 0x6c6c756e
[read_HNSW NL v4 FIX] Detected FileIOReader. Neighbors size field offset: 326
[read_HNSW NL v4] Reading neighbors data into memory.
[read_HNSW NL v4] Read neighbors data, size: 20
[read_HNSW NL v4] Finished reading metadata and CSR indices.
INFO: Skipping external storage loading, since is_recompute is true.
INFO: Registering backend 'hnsw'


INFO:leann.embedding_server_manager:Embedding server is ready!
INFO:leann.api:  Launching server time: 1.078078269958496 seconds
INFO:leann.embedding_server_manager:Existing server process (PID 4574) is compatible
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: facebook/contriever
INFO:leann.api:  Generated embedding shape: (1, 768)
INFO:leann.api:  Embedding time: 2.9307072162628174 seconds


ZmqDistanceComputer initialized: d=768, metric=0


INFO:leann.api:  Search time: 0.27327895164489746 seconds
INFO:leann.api:  Backend returned: labels=2 results
INFO:leann.api:  Processing 2 passage IDs:
INFO:leann.api:    1. passage_id='0' -> SUCCESS: C# is a powerful programming language and it is good at game development...
INFO:leann.api:    2. passage_id='1' -> SUCCESS: Python is a powerful programming language and it is good at machine learning tasks...
INFO:leann.api:  Final enriched results: 2 passages


[SearchResult(id='0', score=np.float32(0.9874103), text='C# is a powerful programming language and it is good at game development', metadata={}),
 SearchResult(id='1', score=np.float32(0.8922168), text='Python is a powerful programming language and it is good at machine learning tasks', metadata={})]

## Chat with LEANN using retrieved results

In [ ]:
from leann.api import LeannChat

llm_config = {
    "type": "hf",
    "model": "Qwen/Qwen3-0.6B",
}

chat = LeannChat(index_path="knowledge.leann", llm_config=llm_config)
response = chat.ask(
    "Compare the two retrieved programming languages and tell me their advantages.",
    top_k=2,
    llm_kwargs={"max_tokens": 128}
)
response

INFO:leann.chat:Attempting to create LLM of type='hf' with model='Qwen/Qwen3-0.6B'
INFO:leann.chat:Initializing HFChat with model='Qwen/Qwen3-0.6B'
INFO:leann.chat:MPS is available. Using Apple Silicon GPU.


[read_HNSW - CSR NL v4] Reading metadata & CSR indices (manual offset)...
[read_HNSW NL v4] Read levels vector, size: 5
[read_HNSW NL v4] Reading Compact Storage format indices...
[read_HNSW NL v4] Read compact_level_ptr, size: 10
[read_HNSW NL v4] Read compact_node_offsets, size: 6
[read_HNSW NL v4] Read entry_point: 4, max_level: 0
[read_HNSW NL v4] Read storage fourcc: 0x6c6c756e
[read_HNSW NL v4 FIX] Detected FileIOReader. Neighbors size field offset: 326
[read_HNSW NL v4] Reading neighbors data into memory.
[read_HNSW NL v4] Read neighbors data, size: 20
[read_HNSW NL v4] Finished reading metadata and CSR indices.
INFO: Skipping external storage loading, since is_recompute is true.


INFO:leann.api:🔍 LeannSearcher.search() called:
INFO:leann.api:  Query: 'Compare the two retrieved programming languages and tell me their advantages.'
INFO:leann.api:  Top_k: 2
INFO:leann.api:  Additional kwargs: {}
INFO:leann.embedding_server_manager:Port 5557 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Port 5558 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Port 5559 has incompatible server, trying next port...
INFO:leann.embedding_server_manager:Found compatible server on port 5560
INFO:leann.embedding_server_manager:Using existing compatible server on port 5560
INFO:leann.api:  Launching server time: 0.04932403564453125 seconds
INFO:leann.embedding_server_manager:Found compatible server on port 5560
INFO:leann.embedding_server_manager:Using existing compatible server on port 5560
INFO:leann.api:  Generated embedding shape: (1, 768)
INFO:leann.api:  Embedding time: 0.06902289390563965 seconds
INFO:leann.api:  S

ZmqDistanceComputer initialized: d=768, metric=0


"<think>\n\n</think>\n\nBased on the context provided, here's a comparison of the two retrieved programming languages:\n\n**C#** is known for being a powerful programming language and is well-suited for game development. It is often used in game development and is popular among developers working on Windows applications.\n\n**Python**, on the other hand, is also a powerful language and is well-suited for machine learning tasks. It is widely used for data analysis, scientific computing, and other applications that require handling large datasets or performing complex calculations.\n\n**Advantages**:\n- C#: Strong for game development and cross-platform compatibility.\n- Python: Strong for"